## Plain logistic regression isn't looking promising

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# read functions.
import os
for f in os.listdir('../fun/'): exec(open('../fun/'+f).read())
del f

# Load data
load( '../out/d3-fight-level-standardize-normalize.pkl' )

X = pd.DataFrame(X)
X.columns = cols

# Change winner to binary 1/0:
y[ y == -1 ] = 0
X.shape

(3576, 115)

If we just throw everything in, getting lots of bad p-values (and a pretty bad r-squared).

In [2]:
# Naive throw everything in
logit_model = sm.Logit( y, X )
result = logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.625733
         Iterations 5
                                              Results: Logit
Model:                            Logit                          Pseudo R-squared:               0.020    
Dependent Variable:               Winner                         AIC:                            4701.2415
Date:                             2019-11-20 21:48               BIC:                            5399.8075
No. Observations:                 3576                           Log-Likelihood:                 -2237.6  
Df Model:                         112                            LL-Null:                        -2284.4  
Df Residuals:                     3463                           LLR p-value:                    0.89646  
Converged:                        1.0000                         Scale:                          1.0000   
No. Iterations:                   5.0000                                                

c:\users\bryce\documents\python37\lib\site-packages\statsmodels\base\model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [3]:
# make predictions and check recall, precision, f1 score.

from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score

pred = result.predict()
print( 
    'Mean wins: %s \nMean predict: %s\n' % ( 
    y.mean(),
    pred.mean()
))

# what is our base level if we predict the majority?
print( 'Accuracy predicting all wins:\n')
print( classification_report( 
    y, 
    [ 1 for x in pred ]
))

# what is the outcome of different cutoffs?
print( 'Accuracy with varying cutoffs:\n' )
for i in range(11): 
    
    icutoff = i/10
    
    predwin = [ 1 if x > i/10 else 0 for x in pred ]
    predloss = [ 0 if x > i/10 else 1 for x in pred ]
    
    fscorewin = f1_score( y, predwin )
    fscoreloss = f1_score( ( y == 0 ) * 1, predloss )    
    prec = precision_score( y, predwin )
    recall = recall_score( y, predwin )
    
    print(
        '%s: \t f1-score: %s   \t precision %s   \t recall: %s' % ( 
            i/10, 
            round( (fscorewin + fscoreloss) / 2, 2 ),
            round( prec, 2 ),
            round( recall, 2 )
    ))

Mean wins: 0.6633109619686801 
Mean predict: 0.4936013393769369

Accuracy predicting all wins:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1204
           1       0.66      1.00      0.80      2372

    accuracy                           0.66      3576
   macro avg       0.33      0.50      0.40      3576
weighted avg       0.44      0.66      0.53      3576

Accuracy with varying cutoffs:

0.0: 	 f1-score: 0.4   	 precision 0.66   	 recall: 1.0
0.1: 	 f1-score: 0.4   	 precision 0.66   	 recall: 1.0
0.2: 	 f1-score: 0.47   	 precision 0.68   	 recall: 0.98
0.3: 	 f1-score: 0.59   	 precision 0.71   	 recall: 0.91
0.4: 	 f1-score: 0.65   	 precision 0.77   	 recall: 0.77
0.5: 	 f1-score: 0.64   	 precision 0.83   	 recall: 0.59
0.6: 	 f1-score: 0.56   	 precision 0.89   	 recall: 0.38
0.7: 	 f1-score: 0.45   	 precision 0.94   	 recall: 0.21
0.8: 	 f1-score: 0.34   	 precision 1.0   	 recall: 0.08
0.9: 	 f1-score: 0.26   	 pr

c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Seems like a cutoff of around .5 gives us way above average wins 
while participating in a large number of fights.
We are capturing 71% of the wins (recall) and winning 75% of the time (precision).
Strangely though, we could win 63% of the time and capture 100% of the wins by
always betting to win.
I guess we need to think about betting and what make the most sense.

In [4]:
# train-test split.
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(
    X, y, 
    random_state = 729,
    test_size = 0.25
)

# fit on train and get in-model accuracy.
m = sm.Logit( y_train, X_train ).fit()

Optimization terminated successfully.
         Current function value: 0.616990
         Iterations 6


In [5]:
print( 
    classification_report( 
        y_train, [ 1 if x > 0.5 else 0 for x in m.predict(X_train) ] 
))

              precision    recall  f1-score   support

           0       0.49      0.78      0.61       896
           1       0.85      0.60      0.70      1786

    accuracy                           0.66      2682
   macro avg       0.67      0.69      0.65      2682
weighted avg       0.73      0.66      0.67      2682



In [6]:
print( 
    classification_report( 
        y_test, [ 1 if x > 0.5 else 0 for x in m.predict(X_test) ] 
))

              precision    recall  f1-score   support

           0       0.45      0.72      0.55       308
           1       0.78      0.53      0.63       586

    accuracy                           0.60       894
   macro avg       0.62      0.63      0.59       894
weighted avg       0.67      0.60      0.60       894



Test accuracy is similar to in-model, so there doesn't seem to be overfitting.

f1-score is lower than test on decision tree though.

Remove most of the insignificant features to see if something looks better. The only problem is that there isn't much predictive value regardless.

In [7]:
# prior model with incomplete dataset:

data = pd.read_csv("../out/d1-fight-level-transform.csv", index_col = 0)

# Change winner to binary 1/0:\n",
data.Winner = data.Winner.apply(lambda x: np.where(x == -1, 0, 1))

# Initial features and target\n",
features = pd.Series(data.columns, index = data.columns)
target = "Winner"

# Remove referree, date, location, winner, title_bout, weight_class, no_of_rounds\n",
features.drop(index = ["Referee", "date", "location", "Winner", "title_bout",
                       "weight_class", "no_of_rounds"], inplace = True)

# Lots of win columns\n",
features.drop(index = ["Diff_win_by_Decision_Majority",
                       "Diff_win_by_Decision_Split",
                       "Diff_win_by_Decision_Unanimous",
                       "Diff_win_by_KO/TKO",
                       "Diff_win_by_Submission",
                       "Diff_win_by_TKO_Doctor_Stoppage"], inplace = True)

# Naive throw everything in
logit_model = sm.Logit( data[target], data[features])
result = logit_model.fit()
print(result.summary2())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [8]:
features_adj = features.drop(index = result.summary2().tables[1].index[result.summary2().tables[1]["P>|z|"] > .15])
logit_model = sm.Logit(data[target], data[features_adj])
result = logit_model.fit()
print(result.summary2())

KeyError: "['title_bout' 'weight_class' 'no_of_rounds'\n 'Diff_win_by_Decision_Majority' 'Diff_win_by_Decision_Split'\n 'Diff_win_by_Decision_Unanimous' 'Diff_win_by_KO/TKO'\n 'Diff_win_by_Submission' 'Diff_win_by_TKO_Doctor_Stoppage'\n 'date_dayofmonth' 'Referee_Dan Miragliotta' 'Referee_Herb Dean'\n 'Referee_John McCarthy' 'Referee_Keith Peterson' 'Referee_Leon Roberts'\n 'Referee_Marc Goddard' 'Referee_Mario Yamasaki' 'Referee_Other'\n 'Referee_Steve Mazzagatti' 'Referee_Yves Lavigne'] not found in axis"

Maybe the effects are different by weight class? These results show more promise in some cases (though probably not enough effectiveness for a betting strategy).

In [9]:
classes = pd.DataFrame(data.weight_class.value_counts())
classes.drop(index = classes.index[np.where(classes.weight_class < 100)], inplace = True)

for x in range(len(classes.index)):
    df = data.loc[data.weight_class == classes.index[x]]
    print("Class: " + classes.index[x])
    logit_model = sm.Logit(df[target], df[features_adj])
    result = logit_model.fit()
    print(result.summary2())

TypeError: can only concatenate str (not "numpy.int64") to str